In [1]:
import numpy as np
import pandas as pd
import re
import os
import spacy

import rake_nltk
from rake_nltk import Metric, Rake

r = Rake()
from flashtext import KeywordProcessor
import pytextrank

import spacy

nlp = spacy.load("en_core_web_sm")

from tqdm import tqdm, tqdm_pandas

tqdm(tqdm())
import matplotlib.pyplot as plt

2023-10-07 05:53:41.187107: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-07 05:53:43.659989: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-07 05:53:45.687438: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-07 05:53:45.695532: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
data_train = pd.read_csv(
    "/home/kareem/hacking/research/nlp_projects/repotech/Resume_Matcher/disaster_tweets/train.csv"
)
data_train.shape


(7613, 5)

In [3]:
data_train.head()


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
data_test = pd.read_csv(
    "/home/kareem/hacking/research/nlp_projects/repotech/Resume_Matcher/disaster_tweets/test.csv"
)
data_test.head(2)


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."


#  Data Preprocessing

In [5]:
own_stop_word = ["i", "we", "are", "and"]


# SPACY LEMMA
def spacy_lemma_text(text):
    doc = nlp(text)
    tokens = [tok.lemma_.lower().strip()
              for tok in doc if tok.lemma_ != "-PRON-"]
    tokens = [tok for tok in tokens if tok not in own_stop_word]
    tokens = " ".join(tokens)
    return tokens


# Remove URL
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


In [6]:
data_train["text_clean"] = data_train["text"].apply(remove_URL)
data_train["text_clean"] = data_train["text"].apply(spacy_lemma_text)
print("Training Clenaing - Done")
data_test["text_clean"] = data_test["text"].apply(remove_URL)
data_test["text_clean"] = data_test["text"].apply(spacy_lemma_text)
print("Text Cleaning - Done")

Training Clenaing - Done
Text Cleaning - Done


# Rapid Automatic Keyword Extraction 
**Rake** short for Rapid Automatic Keyword Extraction algorithm, is a domain independent keyword extraction algorithm which tries to determine key phrases in a body of text by analyzing the frequency of word appearance and its co-occurance with other words in the text.
- This is used for extracting and ranking the keywords/phrases out of a document without any other 
 context except for the document itself.





In [7]:
Rake_keywords = []
r = Rake()
r = Rake(min_length=2, max_length=4)

for text in data_train["text_clean"]:
    r.extract_keywords_from_text(text)
    r.get_ranked_phrases()
    Rake_keywords.append(r.get_ranked_phrases())
data_train["Rake_Keyword"] = Rake_keywords


In [8]:
Rake_keywords = []
r = Rake()
r = Rake(min_length=2, max_length=4)

for text in data_test["text_clean"]:
    r.extract_keywords_from_text(text)
    r.get_ranked_phrases()
    Rake_keywords.append(r.get_ranked_phrases())

data_test["Rake_Keyword"] = Rake_keywords


In [9]:
data_train[["text_clean", "Rake_Keyword"]]


,text_clean,Rake_Keyword
0,our deed be the reason of this # earthquake ma...,[earthquake may allah forgive]
1,forest fire near la ronge sask . canada,[]
2,all resident ask to ' shelter in place ' be be...,"[resident ask, place order]"
3,"13,000 people receive # wildfire evacuation or...","[wildfire evacuation order, 000 people receive]"
4,just got send this photo from ruby # alaska as...,"[wildfire pour, got send]"
...,...,...
7608,two giant crane hold a bridge collapse into ne...,"[two giant crane hold, nearby home http ://, b..."
7609,@aria_ahrary @thetawniest the out of control w...,"[control wild fire, northern part, california ..."
7610,m1.94 [ 01:04 utc]?5 km s of volcano hawaii . ...,"[volcano hawaii, http ://]"
7611,police investigate after an e - bike collide w...,"[life threaten injury, police investigate, lit..."


# FlashText 

    Extract keywords
    Replace keywords
    Case Sensitive
    Span of keywords extracted
    Get Extra information with keywords extracted
    No clean name for Keywords
    Add Multiple Keywords simultaneously
    To Remove keywords
    To check Number of terms in KeywordProcessor
    To check if term is present in KeywordProcessor
    Get all keywords in dictionary


In [10]:
from flashtext import KeywordProcessor

Keyword_prodcessor = KeywordProcessor(case_sensitive=False)


In [11]:
keyword_dict = {"excaltor": ["exclators", "exclators"]}
Keyword_prodcessor.add_keywords_from_dict(keyword_dict)


In [12]:
Flash_keywords = []
for i in data_train["text_clean"]:
    Keyword_prodcessor.extract_keywords(i)
    Flash_keywords.append(Keyword_prodcessor.extract_keywords(i))
data_train["Flash_keyword"] = Flash_keywords

In [13]:
Flash_keywords = []
for i in data_test["text_clean"]:
    Keyword_prodcessor.extract_keywords(i)
    Flash_keywords.append(Keyword_prodcessor.extract_keywords(i))
data_test["Flash_keyword"] = Flash_keywords


In [14]:
data_train

,id,keyword,location,text,target,text_clean,Rake_Keyword,Flash_keyword
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deed be the reason of this # earthquake ma...,[earthquake may allah forgive],[]
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask . canada,[],[]
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all resident ask to ' shelter in place ' be be...,"[resident ask, place order]",[]
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive # wildfire evacuation or...","[wildfire evacuation order, 000 people receive]",[]
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got send this photo from ruby # alaska as...,"[wildfire pour, got send]",[]
...,...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant crane hold a bridge collapse into ne...,"[two giant crane hold, nearby home http ://, b...",[]
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,@aria_ahrary @thetawniest the out of control w...,"[control wild fire, northern part, california ...",[]
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,m1.94 [ 01:04 utc]?5 km s of volcano hawaii . ...,"[volcano hawaii, http ://]",[]
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,police investigate after an e - bike collide w...,"[life threaten injury, police investigate, lit...",[]


# Aspect based Opinion Mining

In [17]:
aspect_terms = []
for review in nlp.pipe(data_train.text_clean):
    chunks = [
        (chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == "NOUN"
    ]
    aspect_terms.append(" ".join(chunks))

data_train["Aspect_Terms"] = aspect_terms


In [26]:
aspect_terms = []
for review in nlp.pipe(data_test.text_clean):
    chunks = [
        (chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == "NOUN"
    ]
    aspect_terms.append(" ".join(chunks))

data_test["Aspect_Terms"] = aspect_terms

In [27]:
data_train.Aspect_Terms


0                                  deed reason earthquake
1                                               fire sask
2       resident shelter place officer evacuation shel...
3                                            people order
4                          photo alaska smoke pour school
                              ...                        
7608                                  crane collapse home
7609                                      fire part state
7610                                               hawaii
7611             police collide car portugal rider injury
7612                                                 raze
Name: Aspect_Terms, Length: 7613, dtype: object

# Sentiment Word Extract 

In [29]:
sentiment_terms = []

for review in nlp.pipe(data_train["text_clean"]):
    if review.is_parsed:
        sentiment_terms.append(
            " ".join(
                [
                    token.lemma_
                    for token in review
                    if (
                        not token.is_stop
                        and not token.is_punct
                        and (token.pos_ == "ADJ" or token.pos_ == "VERB")
                    )
                ]
            )
        )
    else:
        sentiment_terms.append("")

data_train["Sentiment_terms"] = sentiment_terms

/tmp/ipykernel_8291/349933707.py:4: DeprecationWarning: [W107] The property `Doc.is_parsed` is deprecated. Use `Doc.has_annotation("DEP")` instead.
  if review.is_parsed:


In [30]:
sentiment_terms = []

for review in nlp.pipe(data_test["text_clean"]):
    if review.is_parsed:
        sentiment_terms.append(
            " ".join(
                [
                    token.lemma_
                    for token in review
                    if (
                        not token.is_stop
                        and not token.is_punct
                        and (token.pos_ == "ADJ" or token.pos_ == "VERB")
                    )
                ]
            )
        )
    else:
        sentiment_terms.append("")

data_test["Sentiment_terms"] = sentiment_terms

/tmp/ipykernel_8291/965964425.py:4: DeprecationWarning: [W107] The property `Doc.is_parsed` is deprecated. Use `Doc.has_annotation("DEP")` instead.
  if review.is_parsed:
